In [19]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms, models
from going_modular import data_setup, engine, model_builder, predictions
from torchinfo import summary
import zipfile
from pathlib import Path
try:
    from torch.utils.tensorboard import SummaryWriter
except:
    !pip install -q tensorboard
    from torch.utils.tensorboard import SummaryWriter

from typing import Dict, List
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(42)
torch.cuda.manual_seed(42)

### Data

In [2]:
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"
if image_path.is_dir():
    print(f"Image_path: {image_path}")
else:
    print("Image_patt does not exist")


Image_path: data\pizza_steak_sushi


### Dataset and dataloader

In [9]:
train_dir = image_path / "train"
test_dir = image_path / "test"

weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
automatic_transforms = weights.transforms()
print(f"Automatically created transforms: {automatic_transforms}")


train_dataloader, test_dataloader, classes = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=automatic_transforms,
    batch_size=32
)

print(train_dataloader, test_dataloader, classes)

Automatically created transforms: ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)
<torch.utils.data.dataloader.DataLoader object at 0x00000224FFDAE4D0> <torch.utils.data.dataloader.DataLoader object at 0x00000224FFDA95D0> ['pizza', 'steak', 'sushi']


### Transfer Learning

In [11]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
model = torchvision.models.efficientnet_b0(weights=weights).to(device)

summary(model, input_size=(32, 3, 224, 224))


Layer (type:depth-idx)                                  Output Shape              Param #
EfficientNet                                            [32, 1000]                --
├─Sequential: 1-1                                       [32, 1280, 7, 7]          --
│    └─Conv2dNormActivation: 2-1                        [32, 32, 112, 112]        --
│    │    └─Conv2d: 3-1                                 [32, 32, 112, 112]        864
│    │    └─BatchNorm2d: 3-2                            [32, 32, 112, 112]        64
│    │    └─SiLU: 3-3                                   [32, 32, 112, 112]        --
│    └─Sequential: 2-2                                  [32, 16, 112, 112]        --
│    │    └─MBConv: 3-4                                 [32, 16, 112, 112]        1,448
│    └─Sequential: 2-3                                  [32, 24, 56, 56]          --
│    │    └─MBConv: 3-5                                 [32, 24, 56, 56]          6,004
│    │    └─MBConv: 3-6                              

In [14]:
for param in model.features.parameters():
    param.requires_grad = False

model.classifier = nn.Sequential(
    nn.Dropout(0.2, inplace=True),
    nn.Linear(in_features=1280, 
              out_features=3, 
              bias=True)
)
model = model.to(device)

summary(model, 
        input_size=(32, 3, 224, 224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
)

Layer (type:depth-idx)                                  Input Shape               Output Shape              Param #                   Trainable
EfficientNet                                            [32, 3, 224, 224]         [32, 3]                   --                        Partial
├─Sequential: 1-1                                       [32, 3, 224, 224]         [32, 1280, 7, 7]          --                        False
│    └─Conv2dNormActivation: 2-1                        [32, 3, 224, 224]         [32, 32, 112, 112]        --                        False
│    │    └─Conv2d: 3-1                                 [32, 3, 224, 224]         [32, 32, 112, 112]        (864)                     False
│    │    └─BatchNorm2d: 3-2                            [32, 32, 112, 112]        [32, 32, 112, 112]        (64)                      False
│    │    └─SiLU: 3-3                                   [32, 32, 112, 112]        [32, 32, 112, 112]        --                        --
│    └─Sequential

### Train the model

In [28]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Create a writer with all default settings
writer = SummaryWriter()

def train(model, train_dataloader, test_dataloader, optimizer, loss_fn, epochs, device):

    results = {"train_loss": [], 
               "test_loss": [], 
               "train_acc": [], 
               "test_acc": []
               }
    
    for epoch in tqdm(range(epochs)):

        train_loss, train_acc = engine.train_step(
            model = model, 
            dataloader= train_dataloader, 
            loss_fn = loss_fn, 
            optimizer = optimizer,
            device = device)
        
        test_loss, test_acc = engine.test_step(model=model,
                                        dataloader=test_dataloader,
                                        loss_fn=loss_fn,
                                        device=device)
        
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        # TensorBoard writer
        writer.add_scalars(main_tag="Loss", 
                          tag_scalar_dict={"train loss": train_loss, "test loss": test_loss},
                          global_step=epoch
                          )
        writer.add_scalars(main_tag="Accuracy",
                          tag_scalar_dict={"train acc": train_acc, "test acc": test_acc},
                          global_step=epoch
                          )
        
        writer.add_graph(model=model, 
                         # Pass in an example input
                         input_to_model=torch.randn(32, 3, 224, 224).to(device))
    
    # Close the writer
    writer.close()
    
    return results



In [29]:
results = train(model=model,
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                optimizer=optimizer,
                loss_fn=loss_fn,
                epochs=5,
                device=device)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.9162 | train_acc: 0.6250 | test_loss: 0.7138 | test_acc: 0.8040


 20%|██        | 1/5 [00:28<01:52, 28.03s/it]

Epoch: 2 | train_loss: 0.7176 | train_acc: 0.8789 | test_loss: 0.7458 | test_acc: 0.7945


 40%|████      | 2/5 [00:55<01:23, 27.77s/it]

...

 60%|██████    | 3/5 [01:28<01:00, 30.27s/it]

Epoch: 4 | train_loss: 0.6242 | train_acc: 0.7969 | test_loss: 0.6331 | test_acc: 0.8466


 80%|████████  | 4/5 [01:57<00:29, 29.65s/it]

Epoch: 5 | train_loss: 0.5998 | train_acc: 0.8086 | test_loss: 0.5292 | test_acc: 0.8551


100%|██████████| 5/5 [02:25<00:00, 29.13s/it]


In [ ]:
%pip install tensorboard
%load_ext tensorboard
%tensorboard --logdir runs

Note: you may need to restart the kernel to use updated packages.
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6007 (pid 9384), started 0:00:13 ago. (Use '!kill 9384' to kill it.)

In [38]:
from going_modular import experiment_tracking

example_writer = experiment_tracking.create_writer(experiment_name="data_10_percent",
                               model_name="effnetb0",
                               extra="5_epochs")

[INFO] Created SummaryWriter, saving to: runs\2025-01-02\data_10_percent\effnetb0\5_epochs...
